In [1]:
import numpy as np
import plotly.express as px
import plotly.io as pio

from src.turbofan import Turbofan
from utils.configs import (
    config_ex22,
    config_ex23,
    config_teste1,
    config_teste2,
    config_turbofan,
)

pio.templates.default = "plotly"

### Validação 1: Exemplo 2.2

In [2]:
# turbofan = Turbofan(config_ex22)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### Teste 1 de PRP-38

In [3]:
# turbofan = Turbofan(config_teste1)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### TG: CFM56-7B27

In [4]:
turbofan = Turbofan(config_turbofan)
turbofan.set_air_flow(100)
rated_thrust = 121.4  # kN
fuel_flow = 1.293  # kg/s
optimization_status = turbofan.calibrate_turbofan(rated_thrust, fuel_flow)
turbofan.print_config()
turbofan.plot_calibration_result(fuel_flow / rated_thrust, np.arange(1200,2000, 0.1))
optimization_status

### Turbofan fora do ponto de projeto

Agora vamos validar como o meta-modelo atua fora do ponto de projeto, confrontando os dados da simulação com os dados reais do motor, obtidos via ICAO.

In [5]:
turbofan = Turbofan(config_ex23)
turbofan.set_sea_level_air_flow(756)
turbofan.save_design_point()
turbofan.print_config()

Salvando ponto de projeto...
--- Configuração do Motor Turbofan ---

[ Condições de Voo e Ambiente ]
Mach de voo                 : 0.000
Altitude                    : 0.000 ft
Temperatura Ambiente (T_a)  : 290.000 K
Pressão Ambiente (P_a)      : 101.630 kPa

[ Parâmetros Operacionais ]
Bypass Ratio (BPR)          : 4.749
Razão de Pressão - Fan (Prf): 1.690
Razão de Pressão - Booster (Pr_bst): 1.000
Razão de Pressão - Comp. (Prc): 17.200
Fração de Hidrogênio        : 0.000
Perda de Pressão (%)        : 5.0%
PCI Querosene               : 45000.0 kJ/kg
PCI Hidrogênio              : 120000.0 kJ/kg
Constante dos Gases (R_ar)  : 288.300 m²/(s²·K)
Calor Específico (Cp)       : 1.110 kJ/(kg·K)
Temp. Entrada Turbina (T04) : 1529.524 K
Vazão de Ar Total           : 753.624 kg/s

[ Propriedades dos Componentes ]
Componente                | Eficiência (eta)     | Gamma          
------------------------------------------------------------------
Inlet                     | 0.970                | 1.

In [6]:
turbofan.update_from_N2(1.0)
turbofan.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 290.000              | 101.630             
8 (Fan)         | 340.436              | 171.755             
3 (Compressor)  | 777.772              | 2954.181            
4 (Câmara)      | 1529.524             | 2806.472            
5 (Turbina Comp.) | 1092.189             | 662.844             
6 (Turbina Fan) | 802.230              | 171.624             

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 0.000 m/s
Velocidade Bocal Quente (u_core): 470.941 m/s
Velocidade Bocal Frio (u_fan)   : 306.169 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01928
Empuxo Específico               : 1.93402 kN.s/kg
Consumo Específico (TSFC)       : 0.00997 kg/(kN.s)
Empuxo Total                    : 253.526 kN
Consumo de Combustível          : 2.527 kg/s

------------------

In [7]:
var_and_unit_n1 = {

}
for variable, unit in var_and_unit_n1.items():
    df = turbofan.get_values_by_changing_param(
        variable,
        "N1",
        np.arange(0.3, 1.05, 0.1)
    )
    px.line(
        df,
        x="N1",
        y=variable,
        title=f"{variable} vs N1",
        labels={"N1": "N1 (%)", variable: unit},
        range_x=[0.2, 1.0],
        width=800,
        height=400,
    ).show()

In [8]:
var_and_unit = {
    # "N1": "",
    # "thrust": "kN",
    # "tsfc": "kg/(kN.s)",
    "prf": "",
    "prc": "",
    "pr": "",
    # "p03": "",
    # "t04": "K",
    "p04": "",
    "p05": "",
    "p06": "",
    # "u_core": "kg/s",
    # "fuel_consumption": "kg/s",
    # "bpr": "",
}
for variable, unit in var_and_unit.items():
    if variable in ["prf", "bpr"]:
        param = "N1"
        range_x = [0.2, 1.0]
    else:
        param = "N2"
        range_x = [0.4, 1.0]
    df = turbofan.get_values_by_changing_param(variable, param)
    px.line(
        df,
        x=param,
        y=variable,
        title=f"{variable} vs {param}",
        labels={param: f"{param} (%)", variable: unit},
        range_x=range_x,
        width=800,
        height=400,
    ).show()